# Projeto 1

Fonte dedados: https://rapidapi.com/mrngstar/api/instagram-scraper-api3

Instagram: findocs (6273301299) - https://www.instagram.com/findocs/


In [ ]:
https://instagram-scraper-api3.p.rapidapi.com/

## Install dependencies

In [1]:
!pip install requests pandas

In [2]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.
!ls

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,224 kB]
Ign:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,149 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,71

In [3]:
!pip install pyspark==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 13.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767580 sha256=981b733e18d3eccda52dc261e53fd8f6450fdc9d18aa7fb15f71191b09f6c11e
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


## Imports dependencies

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import requests
import json
import findspark
import os
from google.colab import userdata
import sys

import pandas as pd
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import from_utc_timestamp

## Set

In [5]:
url = userdata.get('api_url')

influencer = "findocs"

headers = {
	"x-rapidapi-key": userdata.get('x-rapidapi-key'),
	"x-rapidapi-host": "instagram-scraper-api3.p.rapidapi.com"
}

codigo_ultima_postagem = ''

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [6]:
findspark.init()
findspark.find()

# Create a PySpark session
spark = SparkSession \
       .builder \
       .appName("Analise Perfil do Instagram") \
       .getOrCreate()
spark

## Get infos from Instagram

### Get PROFILE

In [30]:
querystring = {"username_or_id": influencer}

method = 'user_info'

response = requests.get(f'{url}{method}',
                          headers=headers,
                          params=querystring)

response_instagram = response.json()

perfil = response_instagram.get("data")

full_name = perfil.get('full_name')
id = perfil.get('id')
biography = perfil.get('biography')
category = perfil.get('category')
bio_links = perfil.get('bio_links')
follower_count = perfil.get('follower_count')
following_count = perfil.get('following_count')
hd_profile_pic_versions_url = perfil.get('hd_profile_pic_versions')[0].get('url')

media_count = perfil.get('media_count')
is_verified = perfil.get('is_verified')
threads_profile_glyph_url = perfil.get('threads_profile_glyph_url')



schema = T.StructType([
  T.StructField("full_name", T.StringType(), nullable=False),
  T.StructField("id", T.StringType(), nullable=True),
  T.StructField("biography", T.StringType(), nullable=True),
  T.StructField("category", T.StringType(), nullable=True),
  T.StructField("bio_links", T.StringType(), nullable=True),
  T.StructField("follower_count", T.StringType(), nullable=True),
  T.StructField("following_count", T.StringType(), nullable=True),
  T.StructField("hd_profile_pic_versions_url", T.StringType(), nullable=True),
  T.StructField("media_count", T.StringType(), nullable=True),
  T.StructField("is_verified", T.StringType(), nullable=True),
  T.StructField("threads_profile_glyph_url", T.StringType(), nullable=True)
])

print(type(full_name))

data = [(full_name,id,biography,category,bio_links,follower_count,following_count,hd_profile_pic_versions_url,media_count,is_verified,threads_profile_glyph_url)]
print(data)

df_profile_spark = spark.createDataFrame(data, schema=schema)

df_profile_spark = df_profile_spark.withColumn('ts_exec', F.current_timestamp())

# # Mostrando o DataFrame
df_profile_spark.write.partitionBy("full_name","ts_exec").mode("append").format('parquet').save('/content/Datalake/Instagram/Perfil_v2/')

df_profile_spark.show(truncate=False)


<class 'str'>
[('FinDocs | Educação Financeira', '6273301299', '🧠 A melhor escola de Educação Financeira do país \n🎯 No Mercado Financeiro desde 2008\n👇🏼 Conheça o INT, a nossa assinatura!', 'Education', [{'link_id': '18012319076260617', 'url': 'https://linktr.ee/findocs', 'lynx_url': 'https://l.instagram.com/?u=https%3A%2F%2Flinktr.ee%2Ffindocs&e=AT2ImZ-fzlMnp52Au9h6d5Hm4EA5gTOp7ZFfZEcxlVbvWHuhUQEVqItPIngUsXF7R8Lgt8sJy-wLW5A3L9ZpZ0_8Lxx9ccF7r_0kKXE', 'link_type': 'external', 'title': 'Conheça nossa assinatura!', 'image_url': '', 'icon_url': '', 'is_pinned': False, 'is_verified': False, 'open_external_url_with_in_app_browser': True}], 196109, 6, 'https://scontent-iad3-1.cdninstagram.com/v/t51.2885-19/69957136_555939868505804_5630875182173257728_n.jpg?stp=dst-jpg_s320x320&_nc_ht=scontent-iad3-1.cdninstagram.com&_nc_cat=108&_nc_ohc=ZP7pBAbsJNMQ7kNvgHM4b0S&_nc_gid=b31e02eed3e24799ba283f328b1e8987&edm=AEF8tYYBAAAA&ccb=7-5&oh=00_AYANXK1WltWGb0qqzaPbiZjblxH6_pA7iPcCZGoHdQZMEg&oe=66E7DA21&_nc

### Get POSTS

In [31]:
querystring = {"username_or_id": influencer,"count":"12"}

method = 'user_posts'

response = requests.get(f'{url}{method}',
                          headers=headers,
                          params=querystring)

response_instagram = response.json()

In [32]:
from array import ArrayType

next_max_id = response_instagram.get('data').get('next_max_id')

items = response_instagram.get('data').get('items')

codigo_ultima_postagem = ''

data_posts = []

data = response_instagram.get('data')

for post in items:
  id = post.get('id')
  code = post.get('code')

  if post.get('timeline_pinned_user_ids') == None:
    if codigo_ultima_postagem == '':
      codigo_ultima_postagem = code

  device_timestamp = post.get('device_timestamp') # Lembrar de converter
  like_and_view_counts_disabled = post.get('like_and_view_counts_disabled') #?

  usertags = post.get('usertags',{}) #@ marcados

  list_marcados = []

  for marcados in usertags.get('in',[]):
    list_marcados.append({f"username" :marcados.get('user').get('username'),"full_name" :marcados.get('user').get('full_name')})

  caption = post.get('caption')
  image_versions2 = post.get('image_versions2')
  product_type = post.get('product_type')
  coauthor_producers = post.get('coauthor_producers')
  like_count = post.get('like_count')
  comment_count = post.get('comment_count')
  reshare_count = post.get('reshare_count') #encaminhamentos
  timeline_pinned_user_ids = post.get('timeline_pinned_user_ids')

  schema = T.StructType([
    T.StructField("id", T.StringType(), nullable=True),
    T.StructField("code", T.StringType(), nullable=True),
    T.StructField("device_timestamp", T.StringType(), nullable=True),
    T.StructField("like_and_view_counts_disabled", T.StringType(), nullable=True),
    T.StructField("caption", T.StringType(), nullable=True),
    T.StructField("image_versions2", T.StringType(), nullable=True),
    T.StructField("product_type", T.StringType(), nullable=True),
    T.StructField("coauthor_producers", T.StringType(), nullable=True),
    T.StructField("like_count", T.StringType(), nullable=True),
    T.StructField("comment_count", T.StringType(), nullable=True),
    T.StructField("reshare_count", T.StringType(), nullable=True),
    T.StructField("timeline_pinned_user_ids", T.StringType(), nullable=True)
    ])

  data_posts.append((id,code,device_timestamp,like_and_view_counts_disabled,caption,image_versions2,product_type,coauthor_producers,like_count,comment_count,reshare_count,timeline_pinned_user_ids))

print(data_posts)

df_posts_spark = spark.createDataFrame(data_posts, schema=schema)

df_posts_spark = df_posts_spark.withColumn('ts_exec', F.current_timestamp())

# # Mostrando o DataFrame
df_posts_spark.write.mode("append").format('parquet').save('/content/Datalake/Instagram/Postagens_v2/')

df_posts_spark.show(truncate=False)


[('3431017561326935240_6273301299', 'C-dbEsxBNzI', 1723229174207, False, {'bit_flags': 0, 'created_at': 1723229188, 'created_at_utc': 1723229188, 'did_report_as_spam': False, 'is_ranked_comment': False, 'pk': '17953891736697037', 'share_enabled': False, 'content_type': 'comment', 'media_id': '3431017561326935240', 'status': 'Active', 'type': 1, 'user_id': '6273301299', 'strong_id__': '17953891736697037', 'has_translation': True, 'text': 'A equipe da FinDocs está disponibilizando um relatório gratuito com a análise de 5 empresas: Vale, Weg, Tim, Apple e Copasa.\n\nPara receber, escreva "eu quero" nos comentários!\n\n🧠 Faça parte do INT, a maior plataforma de educação financeira e investimentos mais completa do Brasil! O link está na bio.', 'user': {'pk': '6273301299', 'pk_id': '6273301299', 'id': '6273301299', 'full_name': 'FinDocs | Educação Financeira', 'is_private': False, 'strong_id__': '6273301299', 'fbid_v2': '17841406398974162', 'username': 'findocs', 'is_verified': False, 'profi

In [10]:
spark.read.parquet('/content/Datalake/Instagram/Postagens_v2/').createOrReplaceTempView('postagens')

spark.sql('select caption,like_count,comment_count  from postagens').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Get COMMENTS

In [ ]:
codigo_ultima_postagem

'C_vcI6OOCTG'

In [33]:
uerystring = {"code_or_id_or_url":codigo_ultima_postagem,"sort_order":"popular","min_id":None}

# "min_id":"teste"
method = 'media_comments'
min_id = None

aux = 1

data_comments = []

while True:
  print('Executando')
  print(aux)
  aux = aux+1
  querystring = {"code_or_id_or_url":codigo_ultima_postagem,"sort_order":"recent","min_id":min_id}
  print(f'parameters: {querystring}')

  response = requests.get(f'{url}{method}',
                        headers=headers,
                        params=querystring)


  response_instagram = response.json()

  array_comments = response_instagram.get('data').get('comments')

  for comment in array_comments:
    pk = comment.get('pk')
    user_id = comment.get('user_id')
    text = comment.get('text')
    comment_like_count = comment.get('comment_like_count',0)
    child_comment_count = comment.get('child_comment_count',0)
    user_username = comment.get('user').get('username')
    user_full_name = comment.get('user').get('full_name')
    user_is_verified = comment.get('user').get('is_verified')
    data_comments.append((pk,user_id,text,comment_like_count,child_comment_count,user_username,user_full_name,user_is_verified))

  print(f'min_id_antes: {min_id}')
  min_id = response_instagram.get('data').get('next_min_id')
  print(f'min_id_depois: {min_id}')


  # Se não houver mais itens a serem buscados, encerra o loop
  if not min_id:
    break

schema = T.StructType([
  T.StructField("pk", T.StringType(), nullable=True),
  T.StructField("user_id", T.StringType(), nullable=True),
  T.StructField("text", T.StringType(), nullable=True),
  T.StructField("comment_like_count", T.StringType(), nullable=True),
  T.StructField("child_comment_count", T.StringType(), nullable=True),
  T.StructField("user_username", T.StringType(), nullable=True),
  T.StructField("user_full_name", T.StringType(), nullable=True),
  T.StructField("user_is_verified", T.StringType(), nullable=True)
])



print(data_comments)

df_comments_spark = spark.createDataFrame(data_comments, schema=schema)

df_comments_spark = df_comments_spark.withColumn('ts_exec', F.current_timestamp())

# # Mostrando o DataFrame
df_comments_spark.write.mode("append").format('parquet').save('/content/Datalake/Instagram/comentarios/')

df_comments_spark.show(truncate=False)

Executando
1
parameters: {'code_or_id_or_url': 'C_yfATNhaMt', 'sort_order': 'recent', 'min_id': None}
min_id_antes: None
min_id_depois: {"server_cursor": "QVFEb1hCQi1FM0ZrU28wVUFHajJRaXo1ekFXN3Q3S3ZuTHlUOFFGQS1tUmdfZEZCdlhUR1Q3eHZVWUpsTUpBVHhmTnlSNHpBbWFwN2tieHh2WUxxeGdzc05XRWJvOUlWSml5WEZLVFN3MGxVMmc=", "is_server_cursor_inverse": true}
Executando
2
parameters: {'code_or_id_or_url': 'C_yfATNhaMt', 'sort_order': 'recent', 'min_id': '{"server_cursor": "QVFEb1hCQi1FM0ZrU28wVUFHajJRaXo1ekFXN3Q3S3ZuTHlUOFFGQS1tUmdfZEZCdlhUR1Q3eHZVWUpsTUpBVHhmTnlSNHpBbWFwN2tieHh2WUxxeGdzc05XRWJvOUlWSml5WEZLVFN3MGxVMmc=", "is_server_cursor_inverse": true}'}
min_id_antes: {"server_cursor": "QVFEb1hCQi1FM0ZrU28wVUFHajJRaXo1ekFXN3Q3S3ZuTHlUOFFGQS1tUmdfZEZCdlhUR1Q3eHZVWUpsTUpBVHhmTnlSNHpBbWFwN2tieHh2WUxxeGdzc05XRWJvOUlWSml5WEZLVFN3MGxVMmc=", "is_server_cursor_inverse": true}
min_id_depois: None
[('18035157389155692', '6817473443', 'Quando a ubi e a EA falirem eu serei feliz', 0, 0, 'j.ruan18', 'Jose Ruan', False

In [12]:
spark.read.parquet('/content/Datalake/Instagram/comentarios/').createOrReplaceTempView('comentarios')

spark.sql('select text from comentarios').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Analytics

In [18]:
spark.read.parquet('/content/Datalake/Instagram/Perfil_v2/')\
  .select('full_name','follower_count', from_utc_timestamp('ts_exec', 'America/Sao_Paulo').alias('ts_exec_brt'))\
  .where("id!='3444000070552872069_482001976'")\
  .orderBy('ts_exec_brt', ascending=True)\
  .show(truncate=False)

+-----------------------------+--------------+-----------------------+
|full_name                    |follower_count|ts_exec_brt            |
+-----------------------------+--------------+-----------------------+
|FinDocs | Educação Financeira|196105        |2024-09-11 18:35:59.717|
+-----------------------------+--------------+-----------------------+



## Salvar o DataFrame e exportar para o Google Sheets:

In [37]:
# Função para converter DataFrame do PySpark para Pandas
def convert_to_pandas(df_spark):
    # Identificar as colunas de timestamp e convertê-las para string
    for col_name, dtype in df_spark.dtypes:
        if dtype == 'timestamp':
            df_spark = df_spark.withColumn(col_name, F.col(col_name).cast('string'))

    # Agora converte o DataFrame do PySpark para Pandas
    df_pandas = df_spark.toPandas()
    return df_pandas

# Aplicando a função para os três DataFrames
df_profile_pandas = convert_to_pandas(df_profile_spark)
df_comments_pandas = convert_to_pandas(df_comments_spark)
df_posts_pandas = convert_to_pandas(df_posts_spark)

In [39]:
df_profile_pandas.head()

,full_name,id,biography,category,bio_links,follower_count,following_count,hd_profile_pic_versions_url,media_count,is_verified,threads_profile_glyph_url,ts_exec
0,FinDocs | Educação Financeira,6273301299,🧠 A melhor escola de Educação Financeira do pa...,Education,"[{icon_url=, link_type=external, open_external...",196109,6,https://scontent-iad3-1.cdninstagram.com/v/t51...,2448,false,https://www.threads.net/@findocs?modal=true&xm...,2024-09-11 22:45:18.454


In [40]:
df_posts_pandas.head()

,id,code,device_timestamp,like_and_view_counts_disabled,caption,image_versions2,product_type,coauthor_producers,like_count,comment_count,reshare_count,timeline_pinned_user_ids,ts_exec
0,3431017561326935240_6273301299,C-dbEsxBNzI,1723229174207,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],337,916,None,[6273301299],2024-09-11 22:47:31.628
1,3166253794648045264_6273301299,Cvwywz6urbQ,1691666740700,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=e15, u...",feed,[],4213,73,None,[6273301299],2024-09-11 22:47:31.628
2,2923649294938795246_6273301299,CiS44sopezu,1662746174806,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],3354,66,None,[6273301299],2024-09-11 22:47:31.628
3,3454960224216392493_6273301299,C_yfATNhaMt,1726083377851,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],511,26,None,None,2024-09-11 22:47:31.628
4,3454796025462826886_6273301299,C_x5q5NJKeG,1726063804125,false,"{private_reply_status=0, share_enabled=false, ...","{candidates=[{width=1080, scans_profile=, url=...",carousel_container,[],508,14,None,None,2024-09-11 22:47:31.628


In [41]:
df_comments_pandas.head()

,pk,user_id,text,comment_like_count,child_comment_count,user_username,user_full_name,user_is_verified,ts_exec
0,18035157389155692,6817473443,Quando a ubi e a EA falirem eu serei feliz,0,0,j.ruan18,Jose Ruan,false,2024-09-11 22:47:31.354
1,18331257937181174,263058173,"Só jogo repetitivo, tá foda! 🤬",0,0,mateuscoutinhoo,Mateus Coutinho,false,2024-09-11 22:47:31.354
2,17903498064041320,67890423067,Quem lacra não lucra,0,0,miniaturasjlle,Miniatuas Joinville,false,2024-09-11 22:47:31.354
3,17995198136680173,44207040982,Nunca ouvi falar😂,0,0,pelegatib3,Pelegati,false,2024-09-11 22:47:31.354
4,17949650546833361,348032431,"Por isso, meus amigos, invistam sempre em seto...",1,0,orealmb,Marcos Borges,false,2024-09-11 22:47:31.354


In [ ]:
# Salvar os DataFrames em CSV
df_profile_pandas.to_csv('/content/métricas_influenciador_perfil.csv', index=False)
df_comments_pandas.to_csv('/content/métricas_influenciador_comentarios.csv', index=False)
df_posts_pandas.to_csv('/content/métricas_influenciador_postagens.csv', index=False)

#### Calcular as métricas essenciais (engajamento, crescimento de seguidores e análise de conteúdo):


In [42]:
# Calcular engajamento para cada postagem
df_posts_pandas['engagement'] = (df_posts_pandas['like_count'].astype(float) + df_posts_pandas['comment_count'].astype(float)) / df_profile_pandas['follower_count'].astype(float) * 100

# Mostrar o engajamento médio
engagement_avg = df_posts_pandas['engagement'].mean()
print(f"Engajamento médio: {engagement_avg:.2f}%")

Engajamento médio: 0.64%


In [48]:
# Assumindo que você tem o número de seguidores em dois momentos
seguidores_anteriores = 190000  # Exemplo de número anterior de seguidores
seguidores_atuais = df_profile_pandas['follower_count'].astype(int).values[0]

# Calcular o crescimento percentual
crescimento_seguidores = ((seguidores_atuais - seguidores_anteriores) / seguidores_anteriores) * 100
print(f"Crescimento de seguidores: {crescimento_seguidores:.2f}%")

Crescimento de seguidores: 3.22%


In [51]:
# Garantir que as colunas 'like_count' e 'comment_count' sejam numéricas
df_posts_pandas['like_count'] = pd.to_numeric(df_posts_pandas['like_count'], errors='coerce')
df_posts_pandas['comment_count'] = pd.to_numeric(df_posts_pandas['comment_count'], errors='coerce')

# Verifique se há valores nulos e substitua-os por 0 ou outro valor apropriado
df_posts_pandas.fillna({'like_count': 0, 'comment_count': 0}, inplace=True)

# Agora, agrupe por tipo de conteúdo e calcule a média
df_content_analysis = df_posts_pandas.groupby('product_type').agg({
    'like_count': 'mean',
    'comment_count': 'mean'
}).reset_index()

# Renomear as colunas para tornar mais claro
df_content_analysis.columns = ['Tipo de Conteúdo', 'Média de Likes', 'Média de Comentários']

print(df_content_analysis)

     Tipo de Conteúdo  Média de Likes  Média de Comentários
0  carousel_container          1200.9                 155.6
1               clips           709.0                  25.0
2                feed          4213.0                  73.0


#### Salvar os dados no drive:

In [52]:
# Defina o caminho da pasta no Google Drive onde deseja salvar os arquivos
caminho_pasta = '/content/drive/MyDrive/Cursos/Dados/Growdev/projetos/projeto_1/csvs/'

# Certifique-se de que a pasta existe (você pode criar a pasta se necessário)
import os

if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)

# Salvar os DataFrames em arquivos CSV
df_profile_pandas.to_csv(os.path.join(caminho_pasta, 'perfil.csv'), index=False)
df_posts_pandas.to_csv(os.path.join(caminho_pasta, 'postagens.csv'), index=False)
df_comments_pandas.to_csv(os.path.join(caminho_pasta, 'comentarios.csv'), index=False)
df_content_analysis.to_csv(os.path.join(caminho_pasta, 'analise_conteudo.csv'), index=False)